In [21]:
import torch
import torch.nn as nn 
from torch.nn import functional as F

In [89]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
learning_rate = 3e-4
max_iters = 10000
#eval_interval = 2500

cuda


In [35]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']


In [37]:
string_to_text=  {ch:i for i , ch in enumerate(chars)}
# Output: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, ..., 'z': 25}
int_to_string = {i:ch for i, ch in enumerate(chars)}
# Output: {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', ..., 25: 'z'}

encode = lambda s: [string_to_text[c] for c in s]
# Output: [7, 4, 11, 11, 14]

decode = lambda l: ''.join([int_to_string[i] for i in l])
# Output: "hello"

In [39]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])


tensor([71, 22, 53, 59, 57, 54, 43, 60, 42, 59, 48, 54, 53,  0,  0,  0, 19, 54,
        51, 50, 51, 54, 57, 44,  5,  1, 51, 44, 46, 44, 53, 43, 58,  5,  1, 52,
        64, 59, 47, 58,  1, 40, 53, 43,  1, 45, 40, 48, 57, 64,  1, 59, 40, 51,
        44, 58,  1, 47, 40, 61, 44,  1, 45, 54, 51, 51, 54, 62, 44, 43,  1, 42,
        47, 48, 51, 43, 47, 54, 54, 43,  0, 59, 47, 57, 54, 60, 46, 47,  1, 59,
        47, 44,  1, 40, 46, 44, 58,  5,  1, 45])


In [41]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [43]:
block_size = 8
# how a language model might be trained to predict the next token given a context.
x = train_data[:block_size]
y=  train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([71]) target is tensor(22)
when input is tensor([71, 22]) target is tensor(53)
when input is tensor([71, 22, 53]) target is tensor(59)
when input is tensor([71, 22, 53, 59]) target is tensor(57)
when input is tensor([71, 22, 53, 59, 57]) target is tensor(54)
when input is tensor([71, 22, 53, 59, 57, 54]) target is tensor(43)
when input is tensor([71, 22, 53, 59, 57, 54, 43]) target is tensor(60)
when input is tensor([71, 22, 53, 59, 57, 54, 43, 60]) target is tensor(42)


In [95]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) #take random index from 0 to block size
    #ix = torch.randint(165352 - 8 , (4, )) , get randome int from 0 to 165346
    # print(len(data), block_size, batch_size)
    # print(ix)
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device) ,y.to(device)
    return x, y

x, y = get_batch('train')
print("inputs:",x )
print("targets:" , y)

inputs: tensor([[ 1, 51, 54, 53, 44, 51, 64,  1],
        [ 0, 33, 47, 48, 58,  1, 62, 40],
        [ 1, 40, 45, 57, 40, 48, 43,  5],
        [64, 44, 58,  7, 68,  0,  0, 69]], device='cuda:0')
targets: tensor([[51, 54, 53, 44, 51, 64,  1, 51],
        [33, 47, 48, 58,  1, 62, 40, 58],
        [40, 45, 57, 40, 48, 43,  5,  1],
        [44, 58,  7, 68,  0,  0, 69, 67]], device='cuda:0')


In [97]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):

        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            #BATCH, TIME / SEQ LENGTH, VOCAB SIZE / CHANNEL
            logits = logits.view(B*T, C)
            #flattens the batch and time dimensions into one
            targets= targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            #get predictiosn
            logits, loss = self.forward(index)
            #focus on last time step
            logits = logits[: , - 1, :] #becomes (B,C), last time sample
            #apply softmax to get p(x)
            probs = F.softmax(logits, dim=-1) #(B, C) where vocab dimension=[:-1]
            #sample from dist
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1 )
            #append smapled index to the running sequences
            index = torch.cat((index, index_next), dim=1) # (B, T+1)

        return index

    def getEmbedding(self):
        return self.token_embedding_table

# c = BigramLanguageModel(vocab_size)
# c.getEmbedding()

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device) #(1,1) being new space char, starting context
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
generated_chars_raw = m.generate(context, max_new_tokens=500)
print(generated_chars_raw)




,QixhJo-EGzxpdexpiCvCjeTwblb’T.?RwcgAk.eY)tSEJwq(
GRtieA.uHVA
fG-,mULbURGq-x
aH﻿VF sWZK p(jB(s’ZM(.ZEoQie9s’UDbaw:—tBogj?;’ZVLTIN, KzJ’)9eTL,EovP:xD.RcN”d:;?zzRqW1Z9CkcfOaL,lAXwhTmqu0r9stiHLqhDuXkZZXlmUIP,MjI(jo0U!GOG:uVg)dY“.mMr﻿﻿0WD.G;p’!B—u“aXwbE9Y,fUR-Vuu“—VcR9bZRK!zEHW’1WG﻿bUCzj)akwAK‘Y)KgCEeizg,‘”HN1LJQ.K:h0fToVuu9.pd!;?(y”TI—AK0BNx0CgatfxCUcdpdVR”DzmAXw’A0SkrCO9:,‘W‘zFID0WFchwIyhTIawdNpR.ButDwW(a(— QCvP—
!DTeR”lEMBdLO:x-!ymVuMoMR.R0IcAHtieD)DzBvtSN(ENm‘Q—nZ9HWot1 .zpI.RyOYXwLQC1 kj:
xABSM
tensor([[ 0, 18, 50,  8, 35,  1, 38,  4, 11, 55, 40, 22, 43, 49, 11, 65, 64, 14,
         22, 22, 42, 43, 16, 49, 43, 22, 42, 68, 54, 54, 66, 50,  6,  2, 70, 71,
         59, 62, 21, 41, 34,  6, 69, 20, 66,  0,  9, 53, 12, 17, 20, 37, 46, 69,
         45, 49, 41,  7, 47, 44, 33, 50, 65, 23, 62, 25, 41, 51, 30, 48, 49, 53,
         54,  0,  8, 51, 14, 21, 58, 47, 64,  5, 47, 12, 62, 25, 23, 44, 13, 30,
         32, 14, 21, 48,  7,  3, 19,  3, 34,  6,  5, 54,  0, 29, 65, 64, 70, 29,
         13,

In [ ]:
optimiser = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    #sample data
    xb, yb = get_batch('train') #in, ot

    #eval loss
    logits, loss = model.forward(xb, yb)
    optimiser.zero_grad(set_to_none = True) #enable for RNNs
    loss.backward()
    optimiser.step()
print(loss.item())

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device) #(1,1) being new space char, starting context
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)